In [16]:
import sys
sys.path.append('../code')
import glob
import config
import helper_functions
from helper_functions import dict_months
import pandas as pd
import time

# Clover data gathering/cleaning

Clover's DB cannot be accessed unless it's called though Clover API which is written in Java<br/>
Created a dictionary with epoch start/end dates for each month of data avaiable from Clover for the restaurant

This 'for loop' will generate URLs to click and download the Clover reports

In [8]:
list_sales_report_urls = []
for month, url in dict_months.items():
    list_sales_report_urls.append(f'{month} = https://www.clover.com/reporting/sales-report?endTimestamp={url[1]}&startTimestamp={url[0]}')
    
# list_sales_report_urls

Script was created to take all the files downloaded and generate a clean csv file called **'net_sales_total.csv'** under the **'data'** folder

In [ ]:
helper_functions.raw_file_clean(dict_months)

Creating a DataFrame to see sales data generated from script above

In [14]:
df = pd.read_csv('../data/net_sales_total.csv')
df['date'] = pd.to_datetime(df['date'])
df

,date,total
0,2015-01-01,0.00
1,2015-01-02,0.00
2,2015-01-03,0.00
3,2015-01-04,0.00
4,2015-01-05,0.00
...,...,...
1821,2019-12-27,2325.50
1822,2019-12-28,3277.75
1823,2019-12-29,3919.75
1824,2019-12-30,2041.50


Visually inspecting for any duplicates

In [12]:
df[df.index.duplicated(keep=False)]

,date,total


# Darksky API data gathering/cleaning

Creates a 'for loop' for calling Darksky API and adds 24hrs for each iteration<br/>
The **call_dark_sky_api(start)** function calls the Darksky API with the epoch time and returns a dataframe<br/>
**list_epoch** will contain a list of each dataframe returned from the function

**Limitation notice:** since Darksky API allows for 1,000 free calls a day. These calls will need to be separated into separate csv files; and then concatenating them to a single csv file containing all days.

In [26]:
list_epoch = []
for epoch in range(config.date_range_010115_12312019):
    if epoch == 0:
        start = config.epoch_start_01012015_030000am
        print(f"epoch time: {start} | local: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start))}")
        list_epoch.append(helper_functions.call_dark_sky_api(start))
    else:
        start += config.epoch_day
        print(f"epoch time: {start} | local: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start))}")
        list_epoch.append(helper_functions.call_dark_sky_api(start))
        
cols = list_epoch[0].columns
df = pd.concat(list_epoch, ignore_index=True)

# list_epoch

In [13]:
df.to_csv('../data/dark_sky_api_csv/01042020-01022021.csv')

In [ ]:
list_csv_files = []
for infile in glob.glob("../data/dark_sky_api_csv/*.csv"):
    dataframe = pd.read_csv(infile, index_col=0)
    list_csv_files.append(dataframe)
    
df = pd.concat(list_csv_files, sort=False, ignore_index=True)
df = df.sort_values(by=['time'])
df = df.reset_index(drop=True)

In [22]:
df.to_csv('../data/dark_sky_api.csv')

In [24]:
df_weather = pd.read_csv('../data/dark_sky_api.csv')
df_weather['time'] = pd.to_datetime(df_weather['time'])
df_weather = df_weather.rename(columns={"time": "date"})
df_weather

,date,summary,icon,sunriseTime,sunsetTime,moonPhase,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,...,visibility,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,ozone
0,2015-01-01,Clear throughout the day.,clear-day,1420114920,1420148460,0.40,0.0000,0.0000,NaN,0.00,...,9.997,23.40,1420113780,39.47,1420144860,20.90,1420110180,35.46,1420147860,NaN
1,2015-01-02,Mostly cloudy throughout the day.,partly-cloudy-day,1420201320,1420234920,0.43,0.0000,0.0000,NaN,0.00,...,9.997,31.96,1420257600,42.82,1420226040,29.76,1420196160,38.64,1420231080,NaN
2,2015-01-03,Rain starting in the afternoon.,rain,1420287720,1420321380,0.46,0.0243,0.1062,NaN,1.00,...,6.432,28.97,1420283040,41.71,1420336080,29.46,1420283040,41.21,1420336080,NaN
3,2015-01-04,"Light rain until morning, starting again in th...",rain,1420374120,1420407840,0.50,0.0139,0.1195,NaN,1.00,...,6.486,38.81,1420347600,55.25,1420415160,39.30,1420347600,54.75,1420415160,NaN
4,2015-01-05,Partly cloudy throughout the day.,partly-cloudy-day,1420460520,1420494300,0.53,0.0000,0.0000,NaN,0.00,...,9.997,21.20,1420516800,49.74,1420434000,14.61,1420516800,45.21,1420434000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,2020-01-09,Clear throughout the day.,clear-day,1578572520,1578606480,0.47,0.0004,0.0016,1.578595e+09,0.05,...,10.000,21.75,1578568260,34.41,1578599460,13.32,1578567120,34.00,1578598620,309.1
1835,2020-01-10,Mostly cloudy throughout the day.,partly-cloudy-day,1578658860,1578692940,0.51,0.0006,0.0018,1.578678e+09,0.23,...,9.850,29.56,1578635640,52.73,1578688080,24.14,1578632400,52.44,1578687840,305.2
1836,2020-01-11,Windy overnight.,rain,1578745260,1578779400,0.55,0.0007,0.0018,1.578729e+09,0.31,...,10.000,45.65,1578719280,68.49,1578767160,44.37,1578728340,67.99,1578767160,291.5
1837,2020-01-12,Light rain and windy in the morning.,rain,1578831660,1578865860,0.58,0.0042,0.0285,1.578827e+09,0.81,...,9.832,44.92,1578888000,66.07,1578845160,42.34,1578888000,65.57,1578845160,293.9
